<a href="https://colab.research.google.com/github/Dhwaj-054/lit-college-codes/blob/main/RSLabExpt2Trial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = {
    'User1': [5, 0, 2, 4],
    'User2': [3, 2, 4, 0],
    'User3': [0, 5, 3, 1],
    'User4': [4, 4,0, 2],
}
items = ['ItemA', 'ItemB', 'ItemC', 'ItemD']
df = pd.DataFrame(data, index=items)

df.head()

,User1,User2,User3,User4
ItemA,5,3,0,4
ItemB,0,2,5,4
ItemC,2,4,3,0
ItemD,4,0,1,2


In [ ]:
mean_ratings = df.mean(axis=0, skipna=True)
df_mc = df.sub(mean_ratings, axis=1)
print(mean_ratings)
print(df_mc)

User1    2.75
User2    2.25
User3    2.25
User4    2.50
dtype: float64
       User1  User2  User3  User4
ItemA   2.25   0.75  -2.25    1.5
ItemB  -2.75  -0.25   2.75    1.5
ItemC  -0.75   1.75   0.75   -2.5
ItemD   1.25  -2.25  -1.25   -0.5


In [ ]:
def mean_centered_cosine(item1, item2):
    mask = (~item1.isna()) & (~item2.isna())
    v1 = item1[mask]
    v2 = item2[mask]
    if len(v1) == 0:
        return 0.0
    num = np.dot(v1, v2)
    denom = np.sqrt(np.dot(v1, v1)) * np.sqrt(np.dot(v2, v2))
    return num/denom if denom != 0 else 0.0

sim_matrix = pd.DataFrame(0, index=items, columns=items)
for i in items:
    for j in items:
        sim_matrix.loc[i, j] = mean_centered_cosine(df_mc.loc[i], df_mc.loc[j])

print(sim_matrix)

          ItemA     ItemB     ItemC     ItemD
ItemA  1.000000 -0.686589 -0.500195  0.305083
ItemB -0.686589  1.000000 -0.004633 -0.582250
ItemC -0.500195 -0.004633  1.000000 -0.486181
ItemD  0.305083 -0.582250 -0.486181  1.000000


/tmp/ipython-input-2725972731.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-0.6865887290837099' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  sim_matrix.loc[i, j] = mean_centered_cosine(df_mc.loc[i], df_mc.loc[j])
/tmp/ipython-input-2725972731.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-0.5001952235383819' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  sim_matrix.loc[i, j] = mean_centered_cosine(df_mc.loc[i], df_mc.loc[j])
/tmp/ipython-input-2725972731.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.30508307783296046' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  sim_matrix.loc[i, j] = mean_

In [ ]:
def predict_rating(user, target_item):
    rated = df[df[user].notnull()].index
    weighted_sum = 0
    total_sim = 0
    for item in rated:
        sim = sim_matrix.loc[target_item, item]
        rating = df.loc[item, user]
        weighted_sum += sim * rating
        total_sim += abs(sim)
    if total_sim == 0:
        return df.mean().mean() # fallback: global mean
    return weighted_sum / total_sim

In [ ]:
def top_n_recommendations(user, n=2):
    unrated = df[df[user].isna()].index
    preds = {item: predict_rating(user, item) for item in unrated}
    return sorted(preds.items(), key=lambda x: x[1], reverse=True)[:n]

In [ ]:
user_id = 'User2'
recommendations = top_n_recommendations(user_id, n=3)
print(f"Top recommendations for {user_id}:")
for item, score in recommendations:
    print(f"{item}: Predicted rating {score:.2f}")

Top recommendations for User2:


In [ ]:
def mae(true, pred):
    true = np.array(true)
    pred = np.array(pred)
    mask = ~np.isnan(true)
    return np.mean(np.abs(true[mask] - pred[mask]))

def rmse(true, pred):
    true = np.array(true)
    pred = np.array(pred)
    mask = ~np.isnan(true)
    return np.sqrt(np.mean((true[mask] - pred[mask])**2))

In [ ]:
def precision_recall_f1(recommended, relevant):
    tp = len(set(recommended) & set(relevant))
    fp = len(set(recommended) - set(relevant))
    fn = len(set(relevant) - set(recommended))
    precision = tp/(tp+fp) if (tp+fp)>0 else 0
    recall = tp/(tp+fn) if (tp+fn)>0 else 0
    f1 = 2*precision*recall/(precision+recall) if (precision+recall)>0 else 0
    return precision, recall, f1

In [ ]:
# Example
true_ratings = [4, 5, 3]
pred_ratings = [3.8, 4.9, 2.7]
print("MAE:", mae(true_ratings, pred_ratings))
print("RMSE:", rmse(true_ratings, pred_ratings))

recommended_items = ['ItemB', 'ItemD']
relevant_items = ['ItemB', 'ItemC']
precision, recall, f1 = precision_recall_f1(recommended_items, relevant_items)
print(f"Precision: {precision:.2f}, Recall: {recall:.2f}, F1-score: {f1:.2f}")


MAE: 0.19999999999999987
RMSE: 0.2160246899469286
Precision: 0.50, Recall: 0.50, F1-score: 0.50


In [ ]:
import pandas as pd
import numpy as np

data = {
    'User1': [5, 3, 2, 4],
    'User2': [3, 2, 4, 1],
    'User3': [4, 5, 3, np.nan],
    'User4': [4, 4, 1, 2],
}
items = ['ItemA', 'ItemB', 'ItemC', 'ItemD']
df = pd.DataFrame(data, index=items)

mean_ratings = df.mean(axis=0, skipna=True)
df_mc = df.sub(mean_ratings, axis=1)

def mean_centered_cosine(item1, item2):
    mask = (~item1.isna()) & (~item2.isna())
    v1 = item1[mask]
    v2 = item2[mask]
    if len(v1) == 0:
        return 0.0
    num = np.dot(v1, v2)
    denom = np.sqrt(np.dot(v1, v1)) * np.sqrt(np.dot(v2, v2))
    return num/denom if denom != 0 else 0.0

sim_matrix = pd.DataFrame(np.nan, index=items, columns=items)
for i in items:
    for j in items:
        sim_matrix.loc[i, j] = mean_centered_cosine(df_mc.loc[i], df_mc.loc[j])

def predict_rating(user, target_item):
    rated = df[df[user].notnull()].index
    weighted_sum = 0
    total_sim = 0
    for item in rated:
        sim = sim_matrix.loc[target_item, item]
        rating = df.loc[item, user]
        weighted_sum += sim * rating
        total_sim += abs(sim)
    if total_sim == 0:
        return df[target_item].mean()  # fallback: item mean
    return weighted_sum / total_sim

def top_n_recommendations(user, n=3):
    unrated = df[df[user].isna()].index
    preds = {item: predict_rating(user, item) for item in unrated}
    return sorted(preds.items(), key=lambda x: x[1], reverse=True)[:n]

user_id = 'User2'
recommendations = top_n_recommendations(user_id, n=3)
print(f"Top recommendations for {user_id}:")
for item, score in recommendations:
    print(f"{item}: Predicted rating {score:.2f}")


Top recommendations for User2:


In [ ]:
import numpy as np
import pandas as pd

# Step 1: Set up data (replace with your own data)
data = {
    'User 1': [3, 1, 0, 5, 2, 4, 0],
    'User 2': [0, 4, 1, 2, 3, 5, 3],
    'User 3': [5, 4, 3, 0, 1, 2, 4],
    'User 4': [2, 3, 5, 1, 0, 0, 1],
    'User 5': [1, 5, 2, 4, 0, 3, 2],
    'User 6': [4, 2, 0, 3, 5, 1, 0],
    'User 7': [0, 1, 4, 0, 2, 5, 3],
    'User 8': [3, 0, 1, 2, 4, 0, 5]
}
items = ['Item A', 'Item B', 'Item C', 'Item D', 'Item E', 'Item F', 'Item G']
df = pd.DataFrame(data, index=items)

# Convert all 0 ratings to np.nan, assuming 0 means 'not rated'
df.replace(0, np.nan, inplace=True)

# Step 2: Mean-center by user (subtract average rating per user from each user's rating)
mean_user_ratings = df.mean(axis=0, skipna=True)
df_mc = df.sub(mean_user_ratings, axis=1)

# Step 3: Compute mean-centered cosine similarity between items
def mean_centered_cosine(itemA, itemB):
    # Find indices with ratings for both items
    mask = (~itemA.isna()) & (~itemB.isna())
    vA = itemA[mask]
    vB = itemB[mask]
    if len(vA) == 0: return 0.0
    num = np.dot(vA, vB)
    denom = np.sqrt(np.dot(vA, vA)) * np.sqrt(np.dot(vB, vB))
    return num / denom if denom != 0 else 0.0

sim_matrix = pd.DataFrame(index=items, columns=items)
for i in items:
    for j in items:
        sim_matrix.loc[i, j] = mean_centered_cosine(df_mc.loc[i], df_mc.loc[j])

sim_matrix = sim_matrix.astype(float)

# Step 4: Predict rating for a user and an unrated item
def predict_rating(user, target_item):
    # Find items rated by the user
    user_ratings = df[user]
    rated_items = user_ratings[user_ratings.notnull()].index
    numerator = 0.0
    denominator = 0.0
    for item in rated_items:
        sim = sim_matrix.loc[target_item, item]
        rating = df.loc[item, user]
        numerator += sim * rating
        denominator += abs(sim)
    # Formula: weighted sum of similar items' ratings / sum of similarity scores
    if denominator == 0:  # If no similar items, use mean of target item's ratings
        return df.loc[target_item].mean(skipna=True)
    return numerator / denominator

# Step 5: Recommend Top-N Items for a User
def top_n_recommendations(user, n=3):
    user_ratings = df[user]
    unrated_items = user_ratings[user_ratings.isna()].index
    preds = {item: predict_rating(user, item) for item in unrated_items}
    top_items = sorted(preds.items(), key=lambda x: x[1], reverse=True)[:n]
    return top_items

# Example: Get recommendations for User 2
user_id = 'User 2'
recommendations = top_n_recommendations(user_id, n=3)
print(f"Top recommendations for {user_id}:")
for item, score in recommendations:
    print(f"{item}: Predicted rating {score:.2f}")

# --- Step 6: Performance metrics (optional) ---
def mae(true, pred):
    true = np.array(true)
    pred = np.array(pred)
    mask = ~np.isnan(true)
    return np.mean(np.abs(true[mask] - pred[mask]))

def rmse(true, pred):
    true = np.array(true)
    pred = np.array(pred)
    mask = ~np.isnan(true)
    return np.sqrt(np.mean((true[mask] - pred[mask]) ** 2))

# Precision/Recall/F1 for Top-N recommendations (requires relevant items known)
def precision_recall_f1(recommended, relevant):
    tp = len(set(recommended) & set(relevant))
    fp = len(set(recommended) - set(relevant))
    fn = len(set(relevant) - set(recommended))
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    return precision, recall, f1

# Example usage for metrics
# true_ratings = [4, 5, 3]
# pred_ratings = [3.8, 4.7, 2.5]
# print("MAE:", mae(true_ratings, pred_ratings))
# print("RMSE:", rmse(true_ratings, pred_ratings))
# recommended_items = ['Item G', 'Item C']
# relevant_items = ['Item G', 'Item B']
# pr, rc, f1 = precision_recall_f1([x[0] for x in recommendations], relevant_items)
# print(f"Precision: {pr:.2f}, Recall: {rc:.2f}, F1-score: {f1:.2f}")


Top recommendations for User 2:
Item A: Predicted rating -2.18


In [ ]:
import pandas as pd
import numpy as np

# Step 1: Data Preparation
# (Replace this small matrix with your real user-item ratings. Use np.nan for unrated items!)
data = {
    'User 1': [3, 1, 0, 5, 2, 4, 0],
    'User 2': [0, 4, 1, 2, 3, 5, 3],
    'User 3': [5, 0, 3, 0, 1, 2, 4],
    'User 4': [2, 3, 5, 1, 0, 0, 1],
    'User 5': [1, 5, 2, 4, 0, 3, 2],
    'User 6': [4, 2, 0, 3, 5, 1, 0],
    'User 7': [0, 1, 4, 0, 2, 5, 3],
    'User 8': [3, 0, 1, 2, 4, 0, 5]
}
items = ['Item A', 'Item B', 'Item C', 'Item D', 'Item E', 'Item F', 'Item G']
df = pd.DataFrame(data, index=items)

# Step 2: Mean-Centering (per user)
mean_user_ratings = df.mean(axis=0, skipna=True)
df_mc = df.sub(mean_user_ratings, axis=1)

# Step 3: Mean-Centered Cosine Similarity
def mean_centered_cosine(itemA, itemB):
    mask = (~itemA.isna()) & (~itemB.isna())
    vA = itemA[mask]
    vB = itemB[mask]
    if len(vA) == 0:
        return 0.0
    numerator = np.dot(vA, vB)
    denominator = np.sqrt(np.dot(vA, vA)) * np.sqrt(np.dot(vB, vB))
    return numerator / denominator if denominator != 0 else 0.0

sim_matrix = pd.DataFrame(index=items, columns=items, dtype=float)
for i in items:
    for j in items:
        sim_matrix.loc[i, j] = mean_centered_cosine(df_mc.loc[i], df_mc.loc[j])

# Step 4: Predict Rating for Unrated Items (weighted sum formula)
def predict_rating(user, target_item):
    user_ratings = df[user]
    rated_items = user_ratings[user_ratings.notnull()].index
    numerator = 0.0
    denominator = 0.0
    for item in rated_items:
        sim = sim_matrix.loc[target_item, item]
        rating = df.loc[item, user]
        numerator += sim * rating
        denominator += abs(sim)
    # Formula (from doc): weighted sum of rated items’ ratings/similarities
    # If denominator is zero, use mean rating for target item as fallback
    if denominator == 0:
        return df.loc[target_item].mean(skipna=True)
    return numerator / denominator

# Step 5: Top-N Recommendations
def top_n_recommendations(user, n=3):
    user_ratings = df[user]
    unrated_items = user_ratings[user_ratings.isna()].index
    preds = {item: predict_rating(user, item) for item in unrated_items}
    ranked = sorted(preds.items(), key=lambda x: x[1], reverse=True)[:n]
    return ranked

# Demo: Recommend for User 2
user_id = 'User 2'
recommendations = top_n_recommendations(user_id, n=3)
print(f"Top recommendations for {user_id}:")
for item, score in recommendations:
    print(f"{item}: Predicted rating {score:.2f}")

# ---- Step 6: Performance Evaluation Metrics ----
# Example ground truth and predictions for one user (for demonstration; replace with your real test)
true_ratings = [3, 5, 2]             # Actual ratings for items predicted
pred_ratings = [2.8, 4.6, 2.3]       # Corresponding predicted ratings

# 1. MAE (Mean Absolute Error)
def mae(true, pred):
    true = np.array(true)
    pred = np.array(pred)
    mask = ~np.isnan(true)
    error = np.mean(np.abs(true[mask] - pred[mask]))
    return error
# Formula: MAE = mean(|y_true - y_pred|)

# 2. RMSE (Root Mean Squared Error)
def rmse(true, pred):
    true = np.array(true)
    pred = np.array(pred)
    mask = ~np.isnan(true)
    error = np.sqrt(np.mean((true[mask] - pred[mask]) ** 2))
    return error
# Formula: RMSE = sqrt(mean((y_true - y_pred)^2))

print("\nPerformance Metrics on Example Data:")
print(f"MAE: {mae(true_ratings, pred_ratings):.3f}")
print(f"RMSE: {rmse(true_ratings, pred_ratings):.3f}")

# 3. Precision, Recall, F1 Score (for Top-N recommendations)
# Need RELEVANT item list for user (which items are truly liked)
recommended_items = [item for item, _ in recommendations]  # Items system recommends
relevant_items = ['Item G', 'Item D', 'Item E']            # Items actually liked by user

def precision_recall_f1(recommended, relevant):
    tp = len(set(recommended) & set(relevant))    # True positives
    fp = len(set(recommended) - set(relevant))    # False positives
    fn = len(set(relevant) - set(recommended))    # False negatives
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall    = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision+recall)>0 else 0
    return precision, recall, f1
# Formula:
# Precision = TP/(TP+FP)
# Recall = TP/(TP+FN)
# F1 = 2*Precision*Recall/(Precision+Recall)

p, r, f = precision_recall_f1(recommended_items, relevant_items)
print(f"Precision: {p:.3f}")
print(f"Recall: {r:.3f}")
print(f"F1 Score: {f:.3f}")


Top recommendations for User 2:

Performance Metrics on Example Data:
MAE: 0.300
RMSE: 0.311
Precision: 0.000
Recall: 0.000
F1 Score: 0.000


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import random

In [ ]:
data = dict()

data["User/Item"] = [f"User {j}" for j in range(10)]
for i in range(10):
  data[f"Item {i}"] = [random.randint(1, 5) for _ in range(10)]
df = pd.DataFrame(data)
df.set_index("User/Item", inplace=True)

df.head()

,Item 0,Item 1,Item 2,Item 3,Item 4,Item 5,Item 6,Item 7,Item 8,Item 9
User/Item,,,,,,,,,,
User 0,5,4,2,2,4,5,5,5,4,4
User 1,4,1,1,1,3,2,2,5,2,4
User 2,2,4,3,5,5,3,2,2,3,3
User 3,2,3,4,3,2,1,5,3,4,2
User 4,5,4,4,4,1,5,4,3,4,5


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

item_user_matrix = df.T  # Transpose the matrix to have items as rows
item_similarity = cosine_similarity(item_user_matrix)
item_similarity_df = pd.DataFrame(item_similarity, index=item_user_matrix.index, columns=item_user_matrix.index)


In [ ]:
item_user_matrix_mean_centered = item_user_matrix.apply(lambda x: x - x.mean(), axis=1)
item_similarity_mean_centered = cosine_similarity(item_user_matrix_mean_centered)
item_similarity_mean_centered_df = pd.DataFrame(item_similarity_mean_centered,
                                                index=item_user_matrix_mean_centered.index,
                                                columns=item_user_matrix_mean_centered.index)


In [ ]:
def predict_ratings(user_ratings, user_similarity):
    item_similarity = np.array(user_similarity)
    ratings = item_similarity.dot(user_ratings) / np.array([np.abs(item_similarity).sum(axis=1)])
    return ratings


In [ ]:
user_1_ratings = item_user_matrix['User 1']  # Get User 1's ratings
predicted_ratings = predict_ratings(user_1_ratings, item_similarity_df)  # Predict ratings
print(predicted_ratings)


[[2.54910861 2.44012474 2.44285729 2.44266095 2.54866378 2.47680804
  2.51193026 2.55175405 2.50675533 2.56722786]]


In [ ]:
user_4_ratings = item_user_matrix['User 4']  # Get User 1's ratings
predicted_ratings = predict_ratings(user_4_ratings, item_similarity_df)  # Predict ratings
print(predicted_ratings)

[[3.93354322 3.91718443 3.93466175 3.87737648 3.82760454 3.96392443
  3.90816739 3.88754541 3.88721677 3.89496648]]


In [ ]:
def rmse(true_ratings, predicted_ratings):
    true_ratings = np.array(true_ratings)
    predicted_ratings = np.array(predicted_ratings)
    mse = np.mean((true_ratings - predicted_ratings) ** 2)
    return np.sqrt(mse)

def mae(true_ratings, predicted_ratings):
    true_ratings = np.array(true_ratings)
    predicted_ratings = np.array(predicted_ratings)
    return np.mean(np.abs(true_ratings - predicted_ratings))


true = [5, 3, 2]
pred = [4.5, 2.9, 2.1]
print("RMSE:", rmse(true, pred))
print("MAE:", mae(true, pred))

RMSE: 0.30000000000000004
MAE: 0.2333333333333334


In [ ]:
def precision_recall_f1(recommended, relevant):
    # recommended: list of items recommended by the system
    # relevant: list of items actually liked by the user
    recommended_set = set(recommended)
    relevant_set = set(relevant)

    true_positives = len(recommended_set & relevant_set)
    false_positives = len(recommended_set - relevant_set)
    false_negatives = len(relevant_set - recommended_set)

    print(false_positives)
    print(true_positives)
    print(false_negatives)


    # Precision formula
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0

    # Recall formula
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0

    # F1 Score formula
    if precision + recall > 0:
        f1_score = 2 * precision * recall / (precision + recall)
    else:
        f1_score = 0

    return precision, recall, f1_score


# Example usage:
recommended = ['Item B', 'Item C']
relevant = [ 'Item D', 'Item C']
p, r, f1 = precision_recall_f1(recommended, relevant)
print("Precision:", p)
print("Recall:", r)
print("F1 Score:", f1)


1
1
1
Precision: 0.5
Recall: 0.5
F1 Score: 0.5


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, precision_score, recall_score, f1_score, accuracy_score

actual_ratings = item_user_matrix["User 1"]

rated_items_mask = actual_ratings > 0
predicted_ratings_flat = predicted_ratings.reshape(-1)
predicted_ratings_filtered = predicted_ratings_flat[rated_items_mask]
actual_ratings = actual_ratings[rated_items_mask]


mse = mean_squared_error(actual_ratings, predicted_ratings_filtered)
mae = mean_absolute_error(actual_ratings, predicted_ratings_filtered)


threshold = 3.8
actual_labels = (actual_ratings >= threshold).astype(int)
predicted_labels = (predicted_ratings_filtered >= threshold).astype(int)

precision = precision_score(actual_labels, predicted_labels)
recall = recall_score(actual_labels, predicted_labels)
f1 = f1_score(actual_labels, predicted_labels)

print(f"MSE: {mse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"Precision (threshold={threshold}): {precision:.4f}")
print(f"Recall (threshold={threshold}): {recall:.4f}")
print(f"F1 Score (threshold={threshold}): {f1:.4f}")

recommended_items_mask = predicted_ratings_flat >= threshold
recommended_items = item_user_matrix.index[recommended_items_mask]


print("\nConclusion:")
print("The model provides predictions with relatively low MSE and MAE, indicating that the predicted ratings are generally close to the actual ratings.")
print(f"With a recommendation threshold of {threshold}, the precision, recall, and F1 scores provide insights into the model's ability to identify items that the user would rate highly.")
print("Based on the predicted ratings and a threshold of 4, the following items are recommended:")
if len(recommended_items) > 0:
    print(", ".join(recommended_items))
else:
    print("No items are recommended based on the current threshold.")
print("Further analysis and tuning may be needed to improve the recommendation performance based on these metrics.")


MSE: 3.8399
MAE: 1.6600
Precision (threshold=3.8): 0.3000
Recall (threshold=3.8): 1.0000
F1 Score (threshold=3.8): 0.4615

Conclusion:
The model provides predictions with relatively low MSE and MAE, indicating that the predicted ratings are generally close to the actual ratings.
With a recommendation threshold of 3.8, the precision, recall, and F1 scores provide insights into the model's ability to identify items that the user would rate highly.
Based on the predicted ratings and a threshold of 4, the following items are recommended:
Item 0, Item 1, Item 2, Item 3, Item 4, Item 5, Item 6, Item 7, Item 8, Item 9
Further analysis and tuning may be needed to improve the recommendation performance based on these metrics.
